# Variational Inference

### Estimation of Transition matrix and Emission parameters for each state

Libraries:

In [1]:
import numpy as np
import torch
import pyro
from pyro import poutine
import pyro.distributions as dist
from pyro.infer import SVI, TraceEnum_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
import pandas as pd
import tqdm
from models.UnivariateHMM import UnivariateHMM
from models.CopulaHMM import CopulaHMM
from utils.CopulaHelpers import *

Global variables:

In [2]:
DATA_DIR="data/"
HIDDEN_STATES = 2
TRAINING_STEPS= 500

Import data:

In [3]:
data = pd.read_csv(f"{DATA_DIR}hulls_df_matchday2_reduced.csv")
data = data.dropna()
# Convert the areas from m^2 to dam^2 for computational reasons
data["HomeHull"]=data["HomeHull"]/100
data["AwayHull"]=data["AwayHull"]/100
data.head()

,Time [s],Period,HomeHull,AwayHull
0,2.0,1.0,5.982854,8.106641
1,4.0,1.0,6.470239,9.445797
2,6.0,1.0,6.687160,10.615702
3,8.0,1.0,6.578269,12.365166
4,10.0,1.0,6.843430,14.701596


## Univariate (separate) models

In [4]:
sequence_X = torch.tensor(data["HomeHull"].values)
sequence_Y = torch.tensor(data["AwayHull"].values)

#### Home Team

In [5]:
pyro.clear_param_store()

In [6]:
# Guide
guide = AutoDelta(poutine.block(UnivariateHMM, expose=["probs_x", "probs_alpha", "probs_beta"]))
# Optimizer
optimizer = Adam({"lr": 0.01})
# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(UnivariateHMM, guide, optimizer, loss=elbo)
# Training
tqdm_bar = tqdm.tqdm(range(TRAINING_STEPS))
for step in tqdm_bar:
    loss = svi.step(sequence_X, HIDDEN_STATES)
    if step % 100 == 0:
         tqdm_bar.set_postfix({'LOSS': loss})

100%|██████████| 500/500 [09:37<00:00,  1.16s/it, LOSS=4.96e+3]


In [7]:
posterior_HomeTeam = guide(sequence_X,HIDDEN_STATES)
posterior_HomeTeam

{'probs_x': tensor([[0.7005, 0.2995],
         [0.0164, 0.9836]], grad_fn=<ExpandBackward0>),
 'probs_alpha': tensor([1.4714, 9.8486], grad_fn=<ExpandBackward0>),
 'probs_beta': tensor([0.2133, 1.0731], grad_fn=<ExpandBackward0>)}

In [8]:
torch.save(posterior_HomeTeam,f"parameters/univariateHMM_matchday2_HomeTeam_{HIDDEN_STATES}states.pt")

#### Away Team

In [9]:
pyro.clear_param_store()

In [10]:
# Guide
guide = AutoDelta(poutine.block(UnivariateHMM, expose=["probs_x", "probs_alpha", "probs_beta"]))
# Optimizer
optimizer = Adam({"lr": 0.01})
# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(UnivariateHMM, guide, optimizer, loss=elbo)
# Training
tqdm_bar = tqdm.tqdm(range(TRAINING_STEPS))
for step in tqdm_bar:
    loss = svi.step(sequence_Y, HIDDEN_STATES)
    if step % 100 == 0:
         tqdm_bar.set_postfix({'LOSS': loss})

100%|██████████| 500/500 [09:26<00:00,  1.13s/it, LOSS=5.1e+3] 


In [11]:
posterior_AwayTeam = guide(sequence_Y,HIDDEN_STATES)
posterior_AwayTeam

{'probs_x': tensor([[0.9800, 0.0200],
         [0.2421, 0.7579]], grad_fn=<ExpandBackward0>),
 'probs_alpha': tensor([7.8944, 1.8286], grad_fn=<ExpandBackward0>),
 'probs_beta': tensor([0.7980, 0.3022], grad_fn=<ExpandBackward0>)}

In [12]:
torch.save(posterior_AwayTeam,f"parameters/univariateHMM_matchday2_AwayTeam_{HIDDEN_STATES}states.pt")

## Bivariate copula model

In [13]:
sequence_XY = torch.tensor(data[["HomeHull","AwayHull"]].values)
sequence_XY.shape

torch.Size([1970, 2])

In [16]:
pyro.clear_param_store()

In [17]:
# Guide
guide = AutoDelta(poutine.block(CopulaHMM, expose=["probs_x",
                                               "probs_alpha1",
                                               "probs_beta1",
                                               "probs_alpha2",
                                               "probs_beta2",
                                               "theta"
                                               ]))
# Optimizer
optimizer = Adam({"lr": 0.01})

# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(CopulaHMM, guide, optimizer, loss=elbo)

# Training
tqdm_bar = tqdm.tqdm(range(TRAINING_STEPS))
for step in tqdm_bar:
    loss = svi.step(sequence_XY, HIDDEN_STATES)
    #if step % 50 == 0:
    tqdm_bar.set_postfix({'LOSS': loss})

100%|██████████| 500/500 [20:16<00:00,  2.43s/it, LOSS=9.54e+3]


In [18]:
posterior = guide(sequence_XY,HIDDEN_STATES)
posterior

{'probs_x': tensor([[0.9460, 0.0540],
         [0.0607, 0.9393]], grad_fn=<ExpandBackward0>),
 'probs_alpha1': tensor([ 8.3456, 10.3970], grad_fn=<ExpandBackward0>),
 'probs_beta1': tensor([1.1144, 1.0510], grad_fn=<ExpandBackward0>),
 'probs_alpha2': tensor([23.0592, 11.0126], grad_fn=<ExpandBackward0>),
 'probs_beta2': tensor([1.9427, 1.6321], grad_fn=<ExpandBackward0>),
 'theta': tensor([0.9042, 0.9583], grad_fn=<ExpandBackward0>)}

In [19]:
torch.save(posterior,f"parameters/CopulaHMM_matchday2_{HIDDEN_STATES}states.pt")